# **Trabalho 2 - Reconhecimento de Padrões**
### Árvore de Decisão
##### Renan Henrique Cardoso - 379013

In [1]:
import pandas as pd                        # Utilizado para converter o .mat e trabalhar também com dataframes
import seaborn as sns                      # Plotagem
import matplotlib.pyplot as plt            # Plotagem
import itertools
import math
pd.options.mode.chained_assignment = None  

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
def log(x):
  return 0 if (x == 0) else math.log(x)

# Todas as amostras de um determinado atributo são dispostas ordenadamente
# em um vetor para extração das extremidades de cada classe (limiares)
def getThresholds(df, attr):
  thresholds = []
  classes = df['Class'].unique().tolist()
  for item in classes:
    thresholds.append(df.query(f'Class == {item}')[f'{attr}'].iloc[0])
    thresholds.append(df.query(f'Class == {item}')[f'{attr}'].iloc[-1])
  thresholds.sort()
  thresholds = thresholds[1:-1]
  return thresholds


def getEntropy(df):
  n_samples = df.shape[0]
  entropy = 0
  classes = df['Class'].unique().tolist()
  for item in classes:
    prob = df.query(f'Class == {item}').shape[0]/n_samples
    entropy = entropy - prob * log(prob)
  return entropy 


def splitDataframe(df, attr, value):
  part1 = df.query(f'{attr} <= {value}')
  part2 = df.query(f'{attr} > {value}')
  return part1, part2


def treeDecision(wines):
  best_entropy_attrs = pd.DataFrame(columns=['attribute', 'split_point', 'entropy'])
  n_wines = wines.shape[0]
  
 
  for attribute in wines.columns[1:]: 
    df_class_attr = wines[['Class',f'{attribute}']]
    df_class_attr = df_class_attr.sort_values(['Class', f'{attribute}'], ascending=[True, True])
    thresholds = getThresholds(df_class_attr, attribute)

    for th in thresholds:
      
      partition1, partition2 = splitDataframe(df_class_attr, attribute, th)
      entropy_part1 = getEntropy(partition1)
      entropy_part2 = getEntropy(partition2)

      entropy_threshold = (partition1.shape[0]/n_wines) * entropy_part1 + (partition2.shape[0]/n_wines) * entropy_part2
      row = { 'attribute': attribute, 'split_point': th, 'entropy': entropy_threshold }
      best_entropy_attrs = best_entropy_attrs.append(row, ignore_index=True)
  
  best_entropy_attr = best_entropy_attrs.sort_values('entropy').groupby('attribute').first()
  best_entropy = best_entropy_attr.sort_values('entropy').iloc[0]
  partition1, partition2 = splitDataframe(wines, best_entropy.name, best_entropy.split_point)



  return partition1, partition2, best_entropy

In [39]:
wines = pd.read_csv("https://raw.githubusercontent.com/cardosorrenan/reconhecimentopadroes-ufc/master/wine.csv")
wines.columns = ['Class', 'Alcohol', 'Acid', 'Ash', 'Alcalinity', 
                  'Magnesium', 'TotPhenols', 'Flavanoids', 'NonflavPhenols', 
                  'Proanth', 'ColIntensity', 'Hue', 'OD', 'Proline']

height = 1
def tree_recursive(dataset, height):
  if (getEntropy(dataset) != 0):
    partition1, partition2, result = treeDecision(dataset)
    print('Height:', height, '---' * height, end=' ')
    print(result.name, end=' / ')
    print('H:', round(result.entropy, 2), end=' / ')
    print('Cut:', result.split_point, end=' / ')
    print('Rows:', dataset.shape[0])
    height += 1
    del partition1[result.name]
    del partition2[result.name]
    result1 = tree_decision_recursive(partition1, height)
    result2 = tree_decision_recursive(partition2, height)
    return result1, result2
  else:
    return 


model = tree_recursive(wines, height)




Height: 1 --- Flavanoids / H: 0.64 / Cut: 1.57 / Rows: 177
Height: 2 ------ ColIntensity / H: 0.14 / Cut: 3.85 / Rows: 62
Height: 3 --------- Acid / H: 0.0 / Cut: 3.43 / Rows: 14
Height: 3 --------- Alcalinity / H: 0.0 / Cut: 16.8 / Rows: 48
Height: 2 ------ Proline / H: 0.22 / Cut: 680.0 / Rows: 115
Height: 3 --------- Alcalinity / H: 0.04 / Cut: 17.5 / Rows: 52
Height: 4 ------------ Acid / H: 0.0 / Cut: 1.41 / Rows: 4
Height: 3 --------- Alcohol / H: 0.07 / Cut: 12.99 / Rows: 63
Height: 4 ------------ ColIntensity / H: 0.0 / Cut: 3.9 / Rows: 8


In [ ]:
attribute = 'ColIntensity'
sns.scatterplot(y=0.1, x=partition1.query("Class == 1")[f'{attribute}'], color='red')
sns.scatterplot(y=0.2, x=partition1.query("Class == 2")[f'{attribute}'], color='blue')
sns.scatterplot(y=0.3, x=partition1.query("Class == 3")[f'{attribute}'], color='green')
plt.ylim(-2, 2)

In [11]:
model

((([], []), ([], [])), ((([], []), []), (([], []), [])))